Nil Agell u172941

Jordi Badia u173484

# PART 1 - Text Processing

In [46]:
# if you do not have 'nltk', the following command should work "python -m pip install nltk"
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jordi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import re
import json

Abrimos el documento json y leemos linea a linea. Una vez leidas las lineas, cargamos el json y podremos comprobar que hay un total de 400 tweets:

In [48]:
docs_path = '/Users/jordi/Documents/UNI/4rt Curs/1r Trim/IRWA/P1-IRWA/tw_hurricane_data.json'
with open(docs_path) as fp:
    lines = fp.readlines()

print('Number of tweets:', len(lines))
lines = [json.loads(l) for l in lines]

Number of tweets: 4000


Tambien abrimos el documento csv, que nos permitirá hacer un map de los tweets id con los document ids.

In [49]:
import pandas as pd
# define the dataset location
filename = 'tweet_document_ids_map.csv'
sep="\t"
# Set Pandas to show all the columns
pd.set_option('display.max_columns', None)
# Read the data as a dataframe
data = pd.read_csv(filename,sep,names=["doc", "id"])
data.head()

C:\Users\jordi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,doc,id
0,doc_1,1575918182698979328
1,doc_2,1575918151862304768
2,doc_3,1575918140839673873
3,doc_4,1575918135009738752
4,doc_5,1575918119251419136


In [50]:
def build_terms(line):
    """
    Preprocess the tweet text removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line=  line.lower() ## Transform in lowercase
    line=  line.split() ## Tokenize the text to get a list of terms
    tweet_text=[]
    for word in line:
        #let's try to maintain the links in the correct format for the last part
        if "https" not in word: #we maintain the # and @ because have relevance and we delete all the punctuation
            word = re.sub(r'[^\w\s#@]','', word)
            word = re.sub(r'_','',word)

        if word:
            tweet_text.append(word) 
            
    line=[word for word in tweet_text if not word in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line=[stemmer.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [52]:
import time
def create_index(tweets,data):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of tweets 
    data -- dataframe for mapping the doc and the tweet
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}  # dictionary to map docs ids to tweets ids
    for tweet in tweets:  # Remember, lines contain a dictionary of a tweet
        tw_id = tweet['id']
        terms = build_terms(tweet['full_text']) # tweet_text
        for i in range(len(data)):
            if data['id'][i] == tw_id:
                doc_id = data['doc'][i]
        
    
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_doc_index = {}

        for position, term in enumerate(terms): # terms contains tweet_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_doc_index)
                # append the position to the corresponding list
                
        ## START CODE
                current_doc_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_doc_index[term]=[doc_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current doc index with the main index
        for term_doc, posting_doc in current_doc_index.items():
            index[term_doc].append(posting_doc)
        
        ## END CODE                    
                    
    return index

Here we create the index and we pass to the function de data of the csv:

In [53]:
start_time = time.time()
index = create_index(lines,data)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 46.67 seconds


We print the index results for the term internet and we can see the doc_id and the corresponding position:

In [55]:
print("Index results for the term 'internet': {}\n".format(index['internet']))
print("First 10 Index results for the term 'internet': \n{}".format(index['internet'][:10]))

Index results for the term 'internet': [['doc_20', array('I', [7])], ['doc_46', array('I', [12])], ['doc_146', array('I', [6])], ['doc_380', array('I', [3])], ['doc_498', array('I', [9])], ['doc_558', array('I', [8])], ['doc_607', array('I', [4])], ['doc_1034', array('I', [7])], ['doc_1470', array('I', [18])], ['doc_1920', array('I', [9])], ['doc_2002', array('I', [3, 22])], ['doc_2756', array('I', [0])], ['doc_2797', array('I', [6])], ['doc_3081', array('I', [11])], ['doc_3223', array('I', [2])], ['doc_3269', array('I', [15])], ['doc_3563', array('I', [13])], ['doc_3797', array('I', [3])], ['doc_3854', array('I', [0])], ['doc_3867', array('I', [12])], ['doc_3929', array('I', [10])], ['doc_3981', array('I', [9])]]

First 10 Index results for the term 'internet': 
[['doc_20', array('I', [7])], ['doc_46', array('I', [12])], ['doc_146', array('I', [6])], ['doc_380', array('I', [3])], ['doc_498', array('I', [9])], ['doc_558', array('I', [8])], ['doc_607', array('I', [4])], ['doc_1034', arr

We reuse the search function, which is temporal because later we will implement a better one:

In [56]:
def search(query, index):
    """
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
    ## START DODE
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            # docs = docs Union term_docs
            docs = docs.union(set(term_docs))
        except:
            #term is not in index
            pass
    docs = list(docs)
    return docs

In [57]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
docs = search(query, index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
    print("document_id= {} ".format(d_id))

Insert your query (i.e.: Computer Science):

internet

Sample of 10 results out of 22 for the searched query:

document_id= doc_3981 
document_id= doc_46 
document_id= doc_20 
document_id= doc_558 
document_id= doc_2756 
document_id= doc_498 
document_id= doc_380 
document_id= doc_1920 
document_id= doc_3269 
document_id= doc_2002 


We create the index tfidf (also provided with the class code ):

In [60]:
def create_index_tfidf(tweets, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    num_documents -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """

    index = defaultdict(list)
    tf = defaultdict(list)  #term frequencies of terms in documents (documents in the same order as in the main index)
    df = defaultdict(int)  #document frequencies of terms in the input document
    idf = defaultdict(float)

    for tweet in tweets:
        tw_id = tweet['id']
        terms = build_terms(tweet['full_text'])
        for i in range(len(data)):
            if data['id'][i] == tw_id:
                doc_id = data['doc'][i]

        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_doc_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}


        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_doc_index = {}

        for position, term in enumerate(terms):  ## terms contains tweet text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_doc_index[term][1].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_doc_index[term] = [doc_id, array('I', [position])]  #'I' indicates unsigned int (int in Python)

        # normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_doc_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_doc_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1]) / norm, 4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] += 1 # increment DF for current term

        #merge the current doc index with the main index
        for term_doc, posting_doc in current_doc_index.items():
            index[term_doc].append(posting_doc)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_documents / df[term])), 4)

    return index, tf, df, idf

We generate the new indexing:

In [61]:
start_time = time.time()
num_documents = len(lines)
index, tf, df, idf = create_index_tfidf(lines, num_documents)
print("Total time to create the TD-IDF index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the TD-IDF index: 282.44 seconds


We rank the documents with the provided function in class:

In [62]:
def rank_documents(terms, docs, index, idf, tf):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies

    
    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    #HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        # TODO: check how to vectorize the query
        # query_vector[termIndex]=idf[term]  # original
        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf

    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot
    
    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)
    #print ('\n'.join(result_docs), '\n')
    return result_docs

We get the ranked list of docs by getting the id of the docs that contain the query term:

In [63]:
def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs = [posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf)
    return ranked_docs

We create an auxiliary function to list the hashtags of a tweet:

In [64]:
def list_hashtags(tweet):
    hashtags = []
    for tag in tweet["entities"]["hashtags"]: #we loop the hashtags of the tweet that we can find in tweet["entities"]["hashtags"]
        hashtags.append("#"+tag["text"])
    return ' '.join(hashtags).strip()

We create a function to display the relevant information of the tweet ( we also added the document to see it works properly):

In [65]:
def tweet_display(tweets, idd,data):
    for i in range(len(data)):
        if data['doc'][i] == idd: #we map the doc_id to the tweet_id
            doc = data['doc'][i]
            tw_id = data['id'][i]
            for tweet in tweets: #we loop all tweets until we find the one with the corresponding tweet_id, then we display
                if (tweet['id']) == tw_id:
                    return"Document : "+ str(doc)+  "|" +" Tweet: " + str(tweet['full_text']) + "|" + "Username: " + str(tweet["user"]["name"]) + "|" + "Date: "+ str(tweet["created_at"]) + "|" + "Hashtags: " + list_hashtags(tweet) + "|" + "Likes: " +  str(tweet["favorite_count"]) + "|" + "Retweets: "+ str(tweet["retweet_count"]) + "|" + "Url: " + "twitter.com/"+str(tweet["user"]["id"])+"/status/"+tweet['id_str']+"\n"


In [66]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
ranked_docs = search_tf_idf(query, index)
top = 10

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print(tweet_display(lines, d_id,data))

Insert your query (i.e.: Computer Science):

internet

Top 10 results out of 22 for the searched query:

Document : doc_2756| Tweet: the internet is gold. #HurricaneIan #tryguys https://t.co/lj0eg8YxnE|Username: Désirée|Date: Fri Sep 30 15:36:25 +0000 2022|Hashtags: #HurricaneIan #tryguys|Likes: 12|Retweets: 0|Url: twitter.com/23288823/status/1575872198698418176

Document : doc_2797| Tweet: #HurricaneIan My grandparents house is currently inhabitable. We will be out of power and internet for weeks. https://t.co/oGaIP8BfBN|Username: 🍜|Date: Fri Sep 30 15:34:05 +0000 2022|Hashtags: #HurricaneIan|Likes: 1|Retweets: 1|Url: twitter.com/519197164/status/1575871613890924544

Document : doc_2002| Tweet: I'm back! Sort of. Internet is still iffy, but they're obviously working on it. Lost power Wednesday evening. Returned home from my friend's place yesterday, power finally working this morning. Internet and cell service still spotty. Wow, what an experience! #HurricaneIan|Username: Theo Fenrave